In [3]:
!pip install https://github.com/IBM/coursera/blob/master/systemml-1.3.0-SNAPSHOT-python.tar.gz?raw=true

    100% |################################| 9.9MB 1.1MB/s eta 0:00:01
    100% |################################| 17.3MB 1.0MB/s eta 0:00:01
    100% |################################| 25.2MB 770kB/s eta 0:00:01
    100% |################################| 10.1MB 1.5MB/s eta 0:00:01
    100% |################################| 6.7MB 1.9MB/s eta 0:00:01
    100% |################################| 2.0MB 2.9MB/s eta 0:00:01
    100% |################################| 235kB 3.9MB/s eta 0:00:01
    100% |################################| 512kB 3.8MB/s eta 0:00:01
    100% |################################| 286kB 4.0MB/s eta 0:00:01
  Stored in directory: /home/spark/shared/.cache/pip/wheels/aa/bf/28/4344dd13abd8b9b6cbd4032baf4b851873d2e2288a65631fd2
Successfully built systemml
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.


In [4]:
!ln -s -f ~/user-libs/python3/systemml/systemml-java/systemml-1.3.0-SNAPSHOT-extra.jar ~/user-libs/spark2/systemml-1.3.0-SNAPSHOT-extra.jar
!ln -s -f ~/user-libs/python3/systemml/systemml-java/systemml-1.3.0-SNAPSHOT.jar ~/user-libs/spark2/systemml-1.3.0-SNAPSHOT.jar

# Restart the kernal after running first 2 blocks

In [1]:
from systemml import dml, MLContext
import numpy as np

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190610212224-0001
KERNEL_ID = ac1ff81e-3b5d-4ce4-80d0-d2e582a72cc3


In [2]:
ml = MLContext(spark)

In [3]:
script = """
c = sum(a %*% t(b))
"""

In [6]:
a = np.array([[1, 2, 3]])
b = np.array([[4, 5, 6]])
prog = dml(script).input('a', a).input('b', b).output('c')
c = ml.execute(prog).get('c')
c

SystemML Statistics:
Total execution time:		0.001 sec.
Number of executed Spark inst:	2.




32.0

In [7]:
!pip show systemml

Name: systemml
Version: 1.3.0
Summary: Apache SystemML is a distributed and declarative machine learning platform.
Home-page: http://systemml.apache.org/
Author: Apache SystemML
Author-email: dev@systemml.apache.org
License: Apache 2.0
Location: /home/spark/shared/user-libs/python3
Requires: Pillow, scipy, numpy, scikit-learn, pandas
Required-by: 


In [8]:
sc.version

'2.3.3'

In [10]:
print(ml.info())

Archiver-Version: Plexus Archiver
Artifact-Id: systemml
Build-Jdk: 1.8.0_181
Build-Time: 2019-03-19 23:17:12 UTC
Built-By: npansar
Created-By: Apache Maven 3.5.4
Group-Id: org.apache.systemml
Main-Class: org.apache.sysml.api.DMLScript
Manifest-Version: 1.0
Minimum-Recommended-Spark-Version: 2.3.0
Version: 1.3.0-SNAPSHOT



In [11]:
ml = MLContext(sc)

In [12]:
print(ml.info())

Archiver-Version: Plexus Archiver
Artifact-Id: systemml
Build-Jdk: 1.8.0_181
Build-Time: 2019-03-19 23:17:12 UTC
Built-By: npansar
Created-By: Apache Maven 3.5.4
Group-Id: org.apache.systemml
Main-Class: org.apache.sysml.api.DMLScript
Manifest-Version: 1.0
Minimum-Recommended-Spark-Version: 2.3.0
Version: 1.3.0-SNAPSHOT



In [13]:
script = dml("""
print('Hello World');
""")
ml.execute(script)

Hello World
SystemML Statistics:
Total execution time:		0.001 sec.
Number of executed Spark inst:	0.




MLResults

In [14]:
script = dml("""
s = 'Hello World'
""").output("s")

hello_world_str = ml.execute(script).get("s")

print(hello_world_str)

SystemML Statistics:
Total execution time:		0.000 sec.
Number of executed Spark inst:	0.


Hello World


In [15]:
import sys, os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
plt.switch_backend('agg')

# Example 1: Matrix Multiplication

### SystemML script to generate a random matrix, perform matrix multiplication, and compute the sum of the output

In [17]:
script = """
    X = rand(rows=$nr, cols=1000, sparsity=0.5)
    A = t(X) %*% X
    s = sum(A)
"""
prog = dml(script).input('$nr', 1e6).output('s')
s = ml.execute(prog).get('s')
print(s)

SystemML Statistics:
Total execution time:		43.795 sec.
Number of executed Spark inst:	2.


62600179263.402405


## Load diabetes dataset from scikit-learn

In [18]:
%matplotlib inline

In [31]:
diabetes = datasets.load_diabetes()
diabetes_X = diabetes.data[:, np.newaxis, 2]
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]
diabetes_y_train = np.matrix(diabetes.target[:-20]).T
diabetes_y_test = np.matrix(diabetes.target[-20:]).T

In [33]:
script = """
    # add constant feature to X to model intercept
    ones = matrix(1, rows=nrow(X), cols=1)
    X = cbind(X, ones)
    A = t(X) %*% X
    b = t(X) %*% y
    w = solve(A, b)
    bias = as.scalar(w[nrow(w), 1])
    w = w[1:nrow(w) - 1,]
"""

In [34]:
prog = dml(script).input(X=diabetes_X_train, y=diabetes_y_train).output('w', 'bias')
w, bias = ml.execute(prog).get('w', 'bias')
w = w.toNumPy()

SystemML Statistics:
Total execution time:		0.043 sec.
Number of executed Spark inst:	2.




In [36]:
print((w, bias)) #w = slope, bias = intercept

(array([[938.23786125]]), 152.91886182616167)


# Batch gradient decent

In [37]:
script = """
    ones = matrix(1, rows = nrow(X), cols = 1)
    X = cbind(X, ones)
    max_iter = 100
    w = matrix(0, rows=ncol(X), cols = 1)
    for(i in 1:max_iter){
        XtX = t(X) %*% X
        dw = XtX %*% w - t(X) %*% y
        alpha = (t(dw) %*% dw) / (t(dw) %*% XtX %*% dw)
        w = w - dw * alpha
    }
    bias = as.scalar(w[nrow(w),1])
    w = w[1:nrow(w) -1,]
"""

In [38]:
prog = dml(script).input(X=diabetes_X_train, y=diabetes_y_train).output('w').output('bias')
w, bias = ml.execute(prog).get('w', 'bias')
w = w.toNumPy()

SystemML Statistics:
Total execution time:		0.140 sec.
Number of executed Spark inst:	2.




In [39]:
print(bias)
print(w)

152.9188618261617
[[938.23786125]]


# Conjugate Gradient

In [40]:
script = """
    X = cbind(X, matrix(1, rows=nrow(X), cols=1))
    m = ncol(X);
    i = 1;
    max_iter = 20;
    w = matrix(0, rows = m, cols = 1); # initialize weights to 0
    dw = -t(X) %*% y;
    p = -dw;
    norm_r2 = sum(dw^2);
    for(i in 1:max_iter){
        q = t(X) %*% (X %*% p)
        alpha = norm_r2 / sum(p * q);
        w = w + alpha * p;
        dw = dw + alpha * q;
        old_norm_r2 = norm_r2;
        norm_r2 = sum(dw^2);
        p = -dw + (norm_r2 / old_norm_r2) * p;
        i = i + 1;
    }
    bias = as.scalar(w[nrow(w),1])
    w = w[1:nrow(w) -1,]
"""

In [41]:
prog = dml(script).input(X=diabetes_X_train, y=diabetes_y_train).output('w').output('bias')
w, bias = ml.execute(prog).get('w', 'bias')
w = w.toNumPy()

SystemML Statistics:
Total execution time:		0.016 sec.
Number of executed Spark inst:	2.




In [42]:
print(bias)
print(w)

152.91886182616167
[[938.23786125]]


# Using pre built scripts

In [44]:
from systemml import dmlFromResource
prog = dmlFromResource('scripts/algorithms/LinearRegDS.dml').input(X=diabetes_X_train, y=diabetes_y_train).input('$icpt', 1.0).output('beta_out')
w = ml.execute(prog).get('beta_out')
w = w.toNumPy()
bias = w[1]
print(w)

BEGIN LINEAR REGRESSION SCRIPT
Reading X and Y...
Calling the Direct Solver...
Computing the statistics...
AVG_TOT_Y,153.36255924170615
STDEV_TOT_Y,77.21853383600028
AVG_RES_Y,3.633533705616816E-14
STDEV_RES_Y,63.038506337610244
DISPERSION,3973.853281276927
R2,0.3351312506863875
ADJUSTED_R2,0.33354822985468835
R2_NOBIAS,0.3351312506863875
ADJUSTED_R2_NOBIAS,0.33354822985468835
Writing the output matrix...
END LINEAR REGRESSION SCRIPT
SystemML Statistics:
Total execution time:		0.010 sec.
Number of executed Spark inst:	2.


[[938.23687951]
 [152.91886229]]


# Using scikit-learn

In [45]:
from pyspark.sql import SQLContext
from systemml.mllearn import LinearRegression
sqlCtx = SQLContext(sc)

In [46]:
regr = LinearRegression(sqlCtx)
regr.fit(diabetes_X_train, diabetes_y_train)

BEGIN LINEAR REGRESSION SCRIPT
Reading X and Y...
Running the CG algorithm...
||r|| initial value = 64725.64237405237,  target value = 0.06472564237405237
Iteration 1:  ||r|| / ||r init|| = 0.013822097283108787
Iteration 2:  ||r|| / ||r init|| = 5.369915930350396E-14
The CG algorithm is done.
Computing the statistics...
AVG_TOT_Y,153.36255924170615
STDEV_TOT_Y,77.21853383600028
AVG_RES_Y,-8.227243004822623E-12
STDEV_RES_Y,63.03850633759284
DISPERSION,3973.853281274733
R2,0.33513125068675453
ADJUSTED_R2,0.3335482298550564
R2_NOBIAS,0.33513125068675453
ADJUSTED_R2_NOBIAS,0.3335482298550564
Writing the output matrix...
END LINEAR REGRESSION SCRIPT
SystemML Statistics:
Total execution time:		0.006 sec.
Number of executed Spark inst:	2.




lr

In [47]:
predictions = regr.predict(diabetes_X_test)

SystemML Statistics:
Total execution time:		0.003 sec.
Number of executed Spark inst:	1.


